In [5]:
from hssa import HSSAModel, HSSAConfig

model = HSSAModel.from_pretrained('microsoft/mpnet-base', config=HSSAConfig(max_turn_embeddings=20, casual_utterance_attention=True), device_map='cuda')

/home/ilya/repos/hssa/VENV/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-09-12 22:09:28.286804: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-12 22:09:29.076803: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of HSSAModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.encoder.layer.2.attention.bpooler.dense.bias', 'mpnet.encoder.layer.7.attention.bpooler.dense.bias', 'mpnet.encoder.layer.0.attention.bpooler.dense.weight', 'mpnet.encoder.

In [6]:
from hssa import HSSATokenizer

tok = HSSATokenizer.from_pretrained('microsoft/mpnet-base')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MPNetTokenizer'. 
The class this function is called from is 'HSSATokenizer'.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
tok.additional_special_tokens_ids

[30527, 30528]

In [8]:
model.resize_token_embeddings(len(tok))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 30529. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(30529, 768)

In [9]:
model.embeddings

HSSAEmbeddings(
  (word_embeddings): Embedding(30529, 768)
  (position_embeddings): Embedding(514, 768, padding_idx=1)
  (turn_embeddings): Embedding(20, 768, padding_idx=1)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)

In [10]:
import json

batch = json.load(open('val_batch.json', 'r'))[:5]
tokenized = tok(batch).to('cuda')
tokenized.keys(), tokenized['input_ids'].shape

(dict_keys(['input_ids', 'attention_mask', 'max_dia_len', 'utterance_mask']),
 torch.Size([90, 43]))

In [11]:
model(**tokenized).shape

torch.Size([90, 43, 768])